In [ ]:
from IPython.display import display, HTML, clear_output
import time
import random
import requests
from bs4 import BeautifulSoup
import json

In [ ]:
def send_to_google_form(data_dict, form_url):
    form_id = form_url[34:90]
    view_form_url = f'https://docs.google.com/forms/d/e/{form_id}/viewform'
    post_form_url = f'https://docs.google.com/forms/d/e/{form_id}/formResponse'

    page = requests.get(view_form_url)
    content = BeautifulSoup(page.content, "html.parser").find('script', type='text/javascript')
    content = content.text[27:-1]
    result = json.loads(content)[1][1]
    form_dict = {}
    
    loaded_all = True
    for item in result:
        if item[1] not in data_dict:
            print(f"Form item {item[1]} not found. Data not uploaded.")
            loaded_all = False
            return False
        form_dict[f'entry.{item[4][0][0]}'] = data_dict[item[1]]
    
    post_result = requests.post(post_form_url, data=form_dict)
    return post_result.ok
    
#################################################################################################

def run_math_stroop(question, answer):
    print("\nWhen asked, enter the result of the following calculation:")
    print("\n" + "-"*40) 
    time.sleep(1)

    display_question(question, 2, font_size=30) 

    print("\n" + "-"*40) 
    
    start_time = time.time()
    
    user_answer = get_user_answer()

    while user_answer is None:
        user_answer = get_user_answer()

    end_time = time.time()

    time_taken = end_time - start_time
    
    clear_output(wait=True)
    
    if abs(user_answer - answer) < 0.001:
        print("Well done!")
        time.sleep(1)
        print(f"You took {time_taken:.2f} seconds\n")
        time.sleep(2)
        clear_output(wait=True)
        
        return 1, time_taken
    else:
        clear_output(wait=True)
        print(f"Wrong. The correct answer is {answer}\n")
        time.sleep(2)
        clear_output(wait=True)
        return 0, 0
    
def display_question(question, display_time, font_size=30, color='orange', text_align='left', padding_left='70px'):
    style = f"font-size:{font_size}px; color:{color}; text-align:{text_align}; padding-left:{padding_left};"
    html_question = f"<div style='{style}'>{question}</div>"
    display(HTML(html_question))
    time.sleep(display_time)

def get_user_answer():
    try:
        return float(input("Your answer: "))
    except ValueError:
        print("Invalid input. Please enter a number.")
        return None

def main():
    print("Welcome to the Mathematics Ability test!")
    time.sleep(4)
    clear_output()

    data_consent_info = """DATA CONSENT INFORMATION:

    Please read:

    We wish to record your response data
    to an anonymised public data repository. Your data will be used for educational teaching purposes
    practising data anaylsis and visualisation.

    Please type yes in the box below if you consent to the upload."""

    print(data_consent_info)
    result = input("> ")

    if result == "yes":
        print("Thank you for your participation.")
    else: 
        # end code execution by raising an exception
        raise(Exception("User did not consent to continue the test."))
    clear_output()
    
    id_instructions = """

    Enter your anonymised ID.

    To generate an anonymous 4-letter unique user identifer please enter:
    - the first two letters of your favourite fruit.
    - the first two letters of your birth city.
    - the first two letters of your favourite vegetable.

    """

    print(id_instructions)
    user_id = input("> ")

    print("User entered id:", user_id)
    clear_output()
    your_age = input("Enter your age:")
    clear_output()
    
    time.sleep(2)
    
    
    questions = ["2 + 3", "9 - 2", "5 * 4", "13 + 19", "27 / 3", "17 + 2", "4 * 6", "12 - 19", "48 / 4", "3 + 8", "9 * 7", "35 - 26", "12 * 6", "42 + 18", "Cubic root of 27", "81 - 22", "13 * 13", "48 / 6", "4 cubed", "5 cubed"]
    answers = [5, 7, 20, 32, 9, 19, 24, -7, 12, 11, 63, 9, 72, 60, 3, 59, 169, 8, 64, 125]
    
    total_score = 0
    total_time = 0
    num_questions = 0
    duration = 180  
    start_time = time.time()

    while (time.time() - start_time) < duration:
        difficulty = num_questions // 10 + 1
        score, time_taken = run_math_stroop(questions[num_questions % len(questions)], answers[num_questions % len(answers)])
        total_score += score
        total_time += time_taken
        num_questions += 1

    average_time = total_time / num_questions if num_questions > 0 else 0
    print(f"Test over. Your score is {total_score}/{num_questions}")
    print(f"Average time per question: {average_time:.2f} seconds")

    score_fraction = f"{total_score}/{num_questions}"

    data_dict = {'average time': average_time,
                'user id': user_id,
                'age': your_age,
                'score': score_fraction}
    form_url = 'https://docs.google.com/forms/d/e/1FAIpQLSfv-_QPfdMg2ijJaRUdNWi1ajTIe1QYDnNXd5A6X0346F4O4A/viewform?usp=sf_link'
    send_to_google_form(data_dict, form_url)

if __name__ == "__main__":
    main()